In [1]:

import os
import cv2
import random
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img


In [2]:
train_data_dir = 'train_2'
Categories = ['freshapples', 'rottenapples']

In [3]:
IMG_SIZE = 224
batch_size = 32
data = []
for category in Categories:
  folder = os.path.join(train_data_dir,category)
  label = Categories.index(category)
  for img in os.listdir(folder):
    img_path = os.path.join(folder,img)
    img_arr = cv2.imread(img_path)
    img_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
    data.append([img_arr,label])
    

In [4]:
random.shuffle(data)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle = True,
    subset='validation') # set as validation data

Found 3229 images belonging to 2 classes.
Found 806 images belonging to 2 classes.


In [6]:
train_generator

In [7]:
feature = []
label = []

for features,labels in data:
  feature.append(features)
  label.append(labels)

In [8]:
feature = np.array(feature)
label = np.array(label)

In [9]:
feature = feature.astype('float')
feature = feature/255

In [ ]:
feature.shape

(4035, 224, 224, 3)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(256,(3,3),activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(128, input_shape = feature.shape[1:], activation = 'relu'))


model.add(Dense(2, activation='softmax'))

model = Sequential()

model.add(Conv2D(32,(3,3),activation = 'relu',padding='same',))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, input_shape = feature.shape[1:], activation = 'relu'))
# model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
model.fit(feature, label, epochs=8, validation_split=0.1)

Epoch 1/8
114/114 [==============================] - 22s 76ms/step - loss: 0.4868 - accuracy: 0.8009 - val_loss: 0.2130 - val_accuracy: 0.9109
Epoch 2/8
114/114 [==============================] - 7s 61ms/step - loss: 0.2204 - accuracy: 0.9163 - val_loss: 0.3672 - val_accuracy: 0.8441
Epoch 3/8
114/114 [==============================] - 7s 61ms/step - loss: 0.2227 - accuracy: 0.9099 - val_loss: 0.1757 - val_accuracy: 0.9455
Epoch 4/8
114/114 [==============================] - 7s 61ms/step - loss: 0.1609 - accuracy: 0.9378 - val_loss: 0.1517 - val_accuracy: 0.9455
Epoch 5/8
114/114 [==============================] - 7s 61ms/step - loss: 0.1805 - accuracy: 0.9298 - val_loss: 0.2585 - val_accuracy: 0.9233
Epoch 6/8
114/114 [==============================] - 7s 61ms/step - loss: 0.1265 - accuracy: 0.9513 - val_loss: 0.1410 - val_accuracy: 0.9505
Epoch 7/8
114/114 [==============================] - 7s 61ms/step - loss: 0.0923 - accuracy: 0.9692 - val_loss: 0.2373 - val_accuracy: 0.9431
Epoch

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 28, 28, 128)     

In [ ]:
model.save('fresh_rotten_apple_model')

INFO:tensorflow:Assets written to: fresh/rotten_apple_model.pb/assets
